In [ ]:
# No caso de acessar via Google Colab
#from google.colab import drive
#drive.mount('/content/drive/')

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px 
# No caso de não ter instalado o plotly.express, rodar o código abaixo:
# pip install plotly_express==0.4.0

In [5]:
pathEquip = "https://github.com/ArtusC/Mapa-equipamentos-radiologicos/blob/main/Equipamentos%20SC%20POR%20CIDADE.csv?raw=true"
pathLL = "https://github.com/ArtusC/Mapa-equipamentos-radiologicos/blob/main/lat_long_municipios.csv?raw=true"

# No caso de acessar via Google Colab
#pathEquip = '/content/drive/My Drive/-- Data Scientist/Projeto equipamentos radiológicos/Equipamentos SC POR CIDADE.csv'
#pathLL = '/content/drive/My Drive/-- Data Scientist/Projeto equipamentos radiológicos/lat_long_municipios.csv'

#CSV Lat e Long = https://github.com/kelvins/Municipios-Brasileiros.git

In [6]:
#DF com informações dos equipamentos no estado de SC
df = pd.read_csv(pathEquip, encoding = "ISO-8859-1", error_bad_lines=False, sep=';')

#DF com informações de latitude e longitude dos municípios do Brasil
dfLL = pd.read_csv(pathLL, encoding = "utf_8", error_bad_lines=False)

In [7]:
#Mantenho somente os munícipios de SC (UF=42) no DF de long e lat
dfLL = dfLL[dfLL.codigo_uf == 42.0]

In [8]:
#Antes de iniciar o loop para manter só as cidades, retiro essa linha específica
#que sei que vai dar problema (len(x)==5 porém não é uma cidade) e reseto o index do DF
df = df.drop(df.index[257])
df = df.reset_index()

In [9]:
#Lista com todos os equipamentos do df
listEquips = list(df.columns[2:20])

#Dicionario do tipo: dict={equip:{cidade:qtdequip}}, 
#desconsiderando os valores null (nesse caso = '-') e nan
dictEquipCidade = {}

for i1, equips in enumerate(listEquips):
    index = df[equips].index.values
    dictEquipCidade[equips] = {}
    #print(df[equips][index])
    for i2, qtdequips in enumerate(df[equips][index]):
        if qtdequips != '-':
            #print(df.Município[index][i2], qtdequips)
            dictEquipCidade[equips][df.Município[index][i2]] = qtdequips

#Dicionario do tipo: dict={'Cidades_SC':[],'Total_Equipamentos':[],'Quantidade_Equipamentos':[]}
#para transformar em df
dictEquipClean = {'Cidades_SC':[], 'Total_Equipamentos':[], 'Quantidade_Equipamentos':[]}
for i1, row in df.iterrows():
    totalEquips=0
    num_tipo_equips=''
    for i2, val2 in enumerate(row.iloc[2:20].items()):
        tipoEquip, qtdEquips = val2[0], val2[1]
        #print(pd.isna(qtdEquips))
        if (qtdEquips != '-') and (pd.isna(qtdEquips)==False):
            #print(qtdEquips)            
            totalEquips += int(qtdEquips)
            num_tipo_equips += qtdEquips+' x '+tipoEquip
            num_tipo_equips +=', '
    num_tipo_equips = num_tipo_equips[:-2]
    dictEquipClean['Cidades_SC'].append(row.Município)
    dictEquipClean['Total_Equipamentos'].append(totalEquips)
    dictEquipClean['Quantidade_Equipamentos'].append(num_tipo_equips)

dfQtdEqp = pd.DataFrame.from_dict(dictEquipClean)

In [10]:
#Adicionando a coluna CIDADE ao df
lstCidades = []

##Limpeza do df

#Limpeza da coluna Municipio para extrair somente as cidades
for i, x in enumerate(df['Município'].str.split()):
  #print(x, i, len(x))
    if len(x)==2:
        lstCidades.append(x[1])
    elif len(x)==3:
        lstCidades.append(x[1]+" "+x[2])
    elif len(x)==4:
        lstCidades.append(x[1]+" "+x[2]+" "+x[3])
    elif len(x)==5:
        lstCidades.append(x[1]+" "+x[2]+" "+x[3]+" "+x[4])
    else:
    #print(x, i, len(x))
        df = df.drop(index=i)
    #lstCidades.append("")

df['Cidade']=lstCidades

# Com a coluna Cidade adicionada, comparo as cidades com o dfLL e adiciono ao df as colunas lat e long
# Adiciono também o total de equipamentos e a quantidade de equipamentos
dfC = pd.DataFrame(df['Cidade']) #metodo MAP não serve em series

df['Latitude'] = dfC['Cidade'].map(dfLL.drop_duplicates('nome').set_index('nome')['latitude'])
df['Longitude'] = dfC['Cidade'].map(dfLL.drop_duplicates('nome').set_index('nome')['longitude'])
df['Código UF'] = dfC['Cidade'].map(dfLL.drop_duplicates('nome').set_index('nome')['codigo_uf'])

df['Total Equipamentos'] = df['Cidade'].index.to_series().map(dfQtdEqp['Total_Equipamentos'])
df['Quantidade Equipamentos'] = df['Cidade'].index.to_series().map(dfQtdEqp['Quantidade_Equipamentos'])

In [12]:
# Visualização do mapa
token = 'pk.eyJ1IjoiYXJ0dXNjIiwiYSI6ImNrZnBuNDExOTAwOHIycW9jamFnaThwaTAifQ.xbicttjuyTvaI0Nl2xqI-g' 

fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", color="Cidade", size="Total Equipamentos",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=20, zoom=6)

fig.update_layout(mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()